# set display

In [39]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# import data

In [40]:
import pandas as pd
df = pd.read_csv("data/primary_data.csv")

# processing data
- renamed hisp to hispanic in other_race
- change other(3) in race according to other race
    - if yes(1) in hispanic and unknown in other_race change other_race according to hispanic
    - else UNKNOWN
- change other(71) in diagnosis according to other_diagnosis
- change other(71) in diagnosis_2 according to to other_diagnosis_2
- change nan in body_part_2, diagnosis_2 to no
- remove unused cols: other_race, other_diagnosis, other_diagnosis_2

In [42]:
def fill_diagnosis(diagnosis, other_diagnosis):
    if diagnosis == 71:
        return other_diagnosis
    else:
        return diagnosis

In [15]:
def fill_race(race, other_race, hispanic):
    if race == 3:
        if hispanic == 1 and other_race== "UNKNOWN":
            return "HISPANIC"
        elif hispanic == 0 and other_race== "UNKNOWN":
            return "UNKNOWN"
        else:
            return other_race
    else:
        return race

In [43]:
df["other_race"] =  df["other_race"].apply(lambda race: "HISPANIC" if race == "HISP" else race)
df["race"] = df[["race", "other_race", "hispanic"]].apply(lambda row: fill_race(*row), axis=1)
df["diagnosis"] = df[["diagnosis", "other_diagnosis"]].apply(lambda row: fill_diagnosis(*row), axis=1)
df["diagnosis_2"] = df[["diagnosis_2", "other_diagnosis_2"]].apply(lambda row: fill_diagnosis(*row), axis=1)
df["body_part_2"] = df["body_part_2"].fillna("no")
df["diagnosis_2"] = df["body_part_2"].fillna("no")
df = df.drop(columns=["other_race", "other_diagnosis", "other_diagnosis_2"])

# mapper from json

In [44]:
import json
f = open("data/variable_mapping.json")
mapper = json.load(f)

In [55]:
import re

def value_mapper(idx, cols):
    if idx in mapper[cols].keys():
        value = mapper[cols][idx]
        return re.sub(r"\d.\s-\s", "", value)
    else:
        return idx

In [57]:
for col in df.columns:
    if col in mapper.keys():
        df[col] = df[col].astype(str)
        df[col] = df[col].apply(value_mapper, cols= col)
    

In [60]:
df.to_csv("data/cleaned_df.csv")